## <center> RecSys. Home Assignment 1</center>

Один из важных навыков для построения рекомендательных систем - это умение корректно считать метрики качества ранжирования.

В этой домашке мы предлагаем вам потренироваться в этом, и имплементировать метрики Precision@k, Recall@k, MNAP@k и NDCG@k по формулам, чтобы дальше переиспользовать при построении рекомендательных моделей. 

Критерии оценивания:
* Что-то пытался сделать, дописал свой код, но ничего не получилось - 1 балл. 
* Не совсем корректная имплементация одной из 4 метрик, прохождение части тестов - 1 балл. 
* Корректная имплементация одной из 4 метрик, прохождение всех тестов - 2 балла. 
* +1 балл, если получится написать Precision@k, Recall@k без циклов.
* +1 балл, если получится написать NDCG@k, MNAP@k без циклов.

Дедлайн сдачи - **10 октября 23:59**. 

Формат сдачи - отправить Jupyter notebook на почту ananyeva.me@gmail.com с темой письма "[RecSys HW1]" и названием файла Name_Surname_HW1.ipynb.  

Удачи!

In [1]:
from typing import NamedTuple, Union
import tests
import torch
from decimal import Decimal
import numpy as np

C:\Users\dimaz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class PrepareTargetResult(NamedTuple):
    values: torch.Tensor
    indices: torch.Tensor


def validate_metric_inputs(output: torch.Tensor, target: torch.Tensor) -> None:
    if output.size() != target.size():
        raise IndexError(
            "Unequal sizes for output and target: "
            f"output - {output.size()}, target - {target.size()}."
        )
    if not (target.eq(0) | target.eq(1)).all():
        raise ValueError(
            "Target contains values outside of 0 and 1." f"\nTarget:\n{target}"
        )


def prepare_target(
    output: torch.Tensor, target: torch.Tensor, return_indices: bool = False
) -> Union[torch.Tensor, PrepareTargetResult]:
    validate_metric_inputs(output, target)
    # Define order by sorted output scores.
    indices = output.argsort(dim=-1, descending=True)
    sorted_target = torch.gather(target, index=indices, dim=-1)
    return (
        PrepareTargetResult(sorted_target, indices) if return_indices else sorted_target
    )


def nan_to_num(tensor: torch.Tensor, nan: float = 0.0) -> torch.Tensor:
    return torch.where(
        torch.isnan(tensor) | torch.isinf(tensor),
        torch.full_like(tensor, fill_value=nan),
        tensor,
    )


def my_tests(cases, metric):
    flg = False
    for index, case in enumerate(cases):
        print(f"\ncase: {index}\n")
        for k in case['topk']:
            result  = metric(case["output"], case["target"], k)
            if result == case["expected"][k]:
                print(k, 'good')
            else:
                print(k, 'ALERT!')
                print(result)
                flg = True
                break
        if flg:
            break

# Precision

$$P@k = \frac{\sum_{i=1}^k [rel_{i}]}{k}$$

In [60]:
def precision(output: torch.Tensor, target: torch.Tensor, topk: int) -> torch.Tensor:
    # output, target ~ (users, items)
    # target_sorted_by_output ~ (users, items)
    target_sorted_by_output = prepare_target(output, target)
    # YOUR CODE HERE
    topk = min(target.shape[1], topk)
    result = torch.div(torch.sum(target_sorted_by_output[:, :topk], 1), topk).mean().item()
    return result

In [61]:
tests.run_precision(precision)

# Recall

$$P@k = \frac{\sum_{i=1}^k [rel_{i}]}{|rel_k|}$$

In [ ]:
def recall(output: torch.Tensor, target: torch.Tensor, topk: int) -> torch.Tensor:
    # output, target ~ (users, items)
    # target_sorted_by_output ~ (users, items)
    target_sorted_by_output = prepare_target(output, target)
    # YOUR CODE HERE
    result = torch.div(torch.sum(target_sorted_by_output[:, :topk], dim=1), torch.sum(target_sorted_by_output, dim=1)).mean()
    if torch.isnan(result):
        result = 0.0
    return result

In [ ]:
tests.run_recall(recall)

# Mean (Normalized) Average Precision


$$AP@k = \frac{\sum_{i=1}^{k} P@i \cdot [rel@ i]} {\sum_{i=1}^{k}[rel@i]}$$
$$MNAP@k = \frac{1}{N} \sum_{k=1}^{N} \frac{1}{min(k, m_u)} AP@k,$$

где $m_u$ - количество items с интеракциями у пользователя $u$ в тестовый период, <br>
$N$ - количество пользователей. 

In [271]:
def mnap(output: torch.Tensor, target: torch.Tensor, topk: int, normalized: bool = True) -> torch.Tensor:
    # output, target ~ (users, items)
    # target_sorted_by_output ~ (users, items)
    target_sorted_by_output = prepare_target(output, target)
    # YOUR CODE HERE
    tens_to_calc = target_sorted_by_output[:, :topk]
    numerator = tens_to_calc.where((tens_to_calc == 0 ), tens_to_calc.cumsum(dim=1))
    denominator = torch.ones(tens_to_calc.shape).cumsum(dim=1)
    result = torch.div(numerator, denominator).mean(dim=1).mean().item()
    # result = np.mean(list(map(lambda x, y: float(x) / float(y), numerator, denominator)))
    return result

In [254]:
cases = [
    {
        "output": torch.Tensor([[0, 0, 0, 0]]),
        "target": torch.Tensor([[0, 0, 0, 0]]),
        "topk": [1, 3, 10, 100],
        "expected": {
            1: 0.0,
            3: 0.0,
            10: 0.0,
            100: 0.0,
        },
    },
    {
        "output": torch.Tensor([[1, 1, 1, 1]]),
        "target": torch.Tensor([[1, 1, 1, 1]]),
        "topk": [1, 3, 10, 100],
        "expected": {
            1: 1.0,
            3: 1.0,
            10: 1.0,
            100: 1.0,
        },
    },
    {
        "output": torch.Tensor([[0, 0, 0, 0]]),
        "target": torch.Tensor([[1, 1, 1, 1]]),
        "topk": [1, 3, 10, 100],
        "expected": {
            1: 1.0,
            3: 1.0,
            10: 1.0,
            100: 1.0,
        },
    },
    {
        "output": torch.Tensor([[1, 1, 1, 1]]),
        "target": torch.Tensor([[0, 0, 0, 0]]),
        "topk": [1, 3, 10, 100],
        "expected": {
            1: 0.0,
            3: 0.0,
            10: 0.0,
            100: 0.0,
        },
    },
    {
        "output": torch.Tensor([[0.5, 0.4, 0.3, 0.2]]),
        "target": torch.Tensor([[1, 0, 1, 0]]),
        "topk": [1, 3, 10, 100],
        "expected": {
            1: 1.0,
            3: (1 + 2 / 3) / 3,
            10: (1 + 2 / 3) / 4,
            100: (1 + 2 / 3) / 4,
        },
    },
    {
        "output": torch.Tensor(
            [
                [9, 5, 3, 0, 7, 4, 0, 0, 6, 0, 0, 0, 0, 0, 0, 1, 8, 2, 0, 10],
                [0, 0, 1, 5, 9, 3, 0, 0, 0, 0, 0, 4, 0, 0, 10, 7, 0, 2, 8, 6],
                [0, 1, 4, 8, 6, 5, 3, 7, 10, 0, 9, 0, 0, 2, 0, 0, 0, 0, 0, 0],
                [7, 8, 0, 0, 1, 0, 4, 0, 10, 0, 0, 6, 0, 0, 0, 9, 2, 3, 5, 0],
            ]
        ),
        "target": torch.Tensor(
            [
                [1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
                [1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0],
                [0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0],
                [0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0],
            ]
        ),
        "topk": [1, 3, 10, 100],
        "expected": {
            1: (0.0 + 1.0 + 0.0 + 0.0) / 4,
            3: (
                (1 / 2 + 2 / 3) / 3
                + 1 / 3
                + (1 / 2) / 3
                + (1 / 3) / 3
            ) / 4,
            10: (
                (1 / 2 + 2 / 3 + 3 / 6 + 4 / 7 + 5 / 8) / 10
                + (1 + 2 / 9 + 3 / 10) / 10
                + (1 / 2 + 2 / 4 + 3 / 6 + 4 / 8 + 5 / 9) / 10
                + (1 / 3 + 2 / 5 + 3 / 8) / 10 
            ) / 4,
            100: (
                (1 / 2 + 2 / 3 + 3 / 6 + 4 / 7 + 5 / 8 + 6 / 12 + 7 / 14 + 8 / 20) / 20
                + (1 + 2 / 9 + 3 / 10 + 4 / 12 + 5 / 14 + 6 / 15 + 7 / 16 + 8 / 17) / 20
                + (1 / 2 + 2 / 4 + 3 / 6 + 4 / 8 + 5 / 9 + 6 / 11 + 7 / 13 + 8 / 16 + 9 / 18 + 10 / 19) / 20
                + (1 / 3 + 2 / 5 + 3 / 8 + 4 / 12 + 5 / 16 + 6 / 17 + 7 / 18 + 8 / 19 + 9 / 20) / 20
            ) / 4
        },
    },
]

In [273]:
my_tests(cases, mnap)


case: 0



IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [239]:
output.ravel()

tensor([0.5000, 0.4000, 0.3000, 0.2000])

In [276]:
topk = 3
case = cases[0]
output = case['output']
target = case['target']
target_sorted_by_output = prepare_target(output, target)
tens_to_calc = target_sorted_by_output[:, :topk]
numerator = tens_to_calc.where((tens_to_calc == 0), tens_to_calc.cumsum(dim=1))
denominator = torch.ones(tens_to_calc.shape).cumsum(dim=1)
torch.div(numerator, denominator).mean(dim=1).mean().item()

0.0

In [267]:
mnap(output, target, topk)

0.2064180611055611

In [270]:
mnap(output, target, topk) - (
                    (1 / 2 + 2 / 3 + 3 / 6 + 4 / 7 + 5 / 8 + 6 / 12 + 7 / 14 + 8 / 20) / 20
                    + (1 + 2 / 9 + 3 / 10 + 4 / 12 + 5 / 14 + 6 / 15 + 7 / 16 + 8 / 17) / 20
                    + (1 / 2 + 2 / 4 + 3 / 6 + 4 / 8 + 5 / 9 + 6 / 11 + 7 / 13 + 8 / 16 + 9 / 18 + 10 / 19) / 20
                    + (1 / 3 + 2 / 5 + 3 / 8 + 4 / 12 + 5 / 16 + 6 / 17 + 7 / 18 + 8 / 19 + 9 / 20) / 20
                ) / 4

0.002459077622583794

In [251]:
0.25 - 0.24999999999999994

5.551115123125783e-17

In [234]:
denominator.ravel()

tensor([1., 1., 1., 1.])

In [236]:
np.mean(list(map(lambda x, y: float(x) / float(y), numerator.ravel(), denominator.ravel())))

0.25

In [232]:
numerator[0]

tensor([0.])

In [229]:
np.mean(list(map(lambda x, y: float(x) / float(y), numerator[0], denominator[0])))

0.0

In [137]:
case

{'output': tensor([[0.5000, 0.4000, 0.3000, 0.2000]]),
 'target': tensor([[1., 0., 1., 0.]]),
 'topk': [1, 3, 10, 100],
 'expected': {1: 1.0,
  3: 0.5555555555555555,
  10: 0.41666666666666663,
  100: 0.41666666666666663}}

In [136]:
target_sorted_by_output

tensor([[1., 0., 1., 0.]])

In [134]:
(
                    (1 / 2 + 2 / 3) / 3
                    + 1 / 3
                    + (1 / 2) / 3
                    + (1 / 3) / 3
                ) / 4

0.24999999999999994

In [43]:
target.shape[1]

4

In [46]:
(1 + 2 / 3) / 3

0.5555555555555555

In [37]:
target_sorted_by_output

tensor([[1., 0., 1., 0.]])

In [272]:
tests.run_map(mnap)
# tests.run_mnap(mnap)

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

# Normalized Dicsounted Cumulative Gain


$$ NDCG @k = \frac{DCG@k}{IDCG@k},$$ где 
$$DCG@k = \sum_{i=1}^{k} \frac{2^{rel_{i}} - 1}{log_2 (i + 1)}$$
$$IDCG@k = \sum_{i=1}^{|rel_{k}|} \frac{2^{rel_{i}} - 1}{log_2 (i + 1)}$$

In [ ]:
def dcg(tensor: torch.Tensor) -> torch.Tensor:
    gains = (2**tensor) - 1
    return gains / torch.log2(torch.arange(0, tensor.size(-1), dtype=torch.float, device=tensor.device) + 2.0)


def ndcg(output: torch.Tensor, target: torch.Tensor, topk: int) -> torch.Tensor:
    # output, target ~ (users, items)
    # target_sorted_by_output ~ (users, items)
    target_sorted_by_output = prepare_target(output, target)
    ideal_target = prepare_target(target, target)
    # YOUR CODE HERE
    return 0

In [ ]:
tests.run_ndcg(ndcg)